In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import LinearRegression

In [53]:
x1=[4,16,25,49,9,100,64,36]
x2=[18,26,9,17,24,47,23,36]
y= (x2 + 4 * np.sqrt(x1))

print(y)

df= pd.DataFrame(columns=['x1','x2','y'], data=[[4,18,26],
                                                [16,26,42],
                                                [25,9,29],
                                                [49,17,45],
                                                [9,24,36],
                                                [100,47,87],
                                                [64,23,55],
                                                [36,36,60]])

df

[26. 42. 29. 45. 36. 87. 55. 60.]


,x1,x2,y
0,4,18,26
1,16,26,42
2,25,9,29
3,49,17,45
4,9,24,36
5,100,47,87
6,64,23,55
7,36,36,60


In [65]:
train = df.iloc[:6]
test = df.iloc[6:]

train_X = train.drop('y', axis=1)
train_y = train.y

test_X = test.drop('y', axis=1)
test_y = test.y

In [66]:
test_X

,x1,x2
6,64,23
7,36,36


In [67]:
#  verificacmos si la regresión lineal puede predecir correctamente

m1 = LinearRegression()
fit1 = m1.fit(train_X, train_y)
preds = fit1.predict(test_X)
print(f"\n{preds}")
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds))}\n")


[54.68988889 55.32337699]
RMSE: 3.314134253707371



In [68]:
# Las predicciones no son malas, pero hagamos algunos cálculos sobre las características de entrada para mejorar
# y = x2 + 4 * sqrt(x1)

train_X.x1 = 4 * np.sqrt(train_X.x1)

test_X.x1 = 4 * np.sqrt(test_X.x1)

print(test_X)

m2 = LinearRegression()

fit2 = m2.fit(train_X, train_y)

preds = fit2.predict(test_X)

print(f"\n{preds}")

print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds))}\n")

     x1  x2
6  32.0  23
7  24.0  36

[55. 60.]
RMSE: 0.0



In [70]:
train = df.iloc[:6]
test = df.iloc[6:]

train_X = train.drop('y', axis=1)
train_y = train.y

test_X = test.drop('y', axis=1)
test_y = test.y

In [71]:
# Creamos una clase transformadora

#__init__: este es el constructor. Se llama cuando se inicializa el pipeline.
#fit(): Se llama cuando hacemos fit en el pipeline.
#transform(): se llama cuando usamos fit o transform en el pipeline

class ExperimentalTransformer(BaseEstimator, TransformerMixin):
  def __init__(self):
    print('\n>>>>>>>init() called.\n')

  def fit(self, X, y = None):
    print('\n>>>>>>>fit() called.\n')
    return self

  def transform(self, X, y = None):
    print('\n>>>>>>>transform() called.\n')
    X_ = X.copy() # creamos una copia para evitar cambios en el conjunto de datos original
    X_.x1 = 4 * np.sqrt(X_.x1)
    return X_

In [72]:
# sin transformación de entrada - para validar que obtenemos los mismos resultados que antes


print("create pipeline 1")
pipe1 = Pipeline(steps=[
                       ('linear_model', LinearRegression())
])

print("fit pipeline 1")
pipe1.fit(train_X, train_y)

print("predict via pipeline 1")
preds1 = pipe1.predict(test_X)

print(f"\n{preds1}")  # should be [13.72113586 16.93334467]
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds1))}\n")

create pipeline 1
fit pipeline 1
predict via pipeline 1

[54.68988889 55.32337699]
RMSE: 3.314134253707371



In [73]:
# con transformation de entradas

print("create pipeline 2")
pipe2 = Pipeline(steps=[
                       ('experimental_trans', ExperimentalTransformer()),    # esto dispara una llamda a __init__
                       ('linear_model', LinearRegression())
])

# una sintaxis alternativa y más corta para hacer lo anterior, sin nombrar cada paso, es:
#pipe2 = make_pipeline(ExperimentalTransformer(), LinearRegression())

print("fit pipeline 2")
pipe2.fit(train_X, train_y)

print("predict via pipeline 2")
preds2 = pipe2.predict(test_X)

print(f"\n{preds2}")  # should be [14. 17.]
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds2))}\n")



# a. __init__ se llama cuando inicializamos pipe2.

# b. fit() y transform() se llaman cuando llamamos fit() con los datos de entrenamiento.
# Esto es necesario para transformar las características de entrada mientras se intenta predecir train_y.

# c. transform() se llama  cuando usamos predict(test_X): las características de prueba de entrada deben tener raíz cuadrada y duplicarse también antes de hacer predicciones.

create pipeline 2

>>>>>>>init() called.

fit pipeline 2

>>>>>>>fit() called.


>>>>>>>transform() called.

predict via pipeline 2

>>>>>>>transform() called.


[55. 60.]
RMSE: 0.0



In [77]:
class ExperimentalTransformer_2(BaseEstimator, TransformerMixin):

  # agregue otro parámetro adicional, solo para mostrar su uso, mientras estamos en ello

  def __init__(self, feature_name, additional_param = "Chontaduro"):
    print('\n>>>>>>>init() ExperimentalTransformer_2 called.\n')
    self.feature_name = feature_name
    self.additional_param = additional_param

  def fit(self, X, y = None):
    print('\n>>>>>>>fit() called.\n')
    print(f'\nadditional param ExperimentalTransformer_2 ~~~~~ {self.additional_param}\n')
    return self

  def transform(self, X, y = None):
    print('\n>>>>>>>transform() called.\n')
    X_ = X.copy() # creating a copy to avoid changes to original dataset
    X_[self.feature_name] = 4 * np.sqrt(X_[self.feature_name])
    return X_

In [78]:
print("create pipeline 2")
pipe2 = Pipeline(steps=[
                       ('experimental_trans', ExperimentalTransformer_2('x1',additional_param='new')),
                       ('linear_model', LinearRegression())
])
print("fit pipeline 2")
pipe2.fit(train_X, train_y)
print("predict via pipeline 2")
preds2 = pipe2.predict(test_X)
print(f"\n{preds2}")
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds2))}\n")

create pipeline 2

>>>>>>>init() ExperimentalTransformer_2 called.

fit pipeline 2

>>>>>>>fit() called.


additional param ExperimentalTransformer_2 ~~~~~ new


>>>>>>>transform() called.

predict via pipeline 2

>>>>>>>transform() called.


[55. 60.]
RMSE: 0.0

